### Code On Nature Inspired Computation - CA1 (Author : Arjun Biswas Student Id: 720036621)

#### Some points to Note:
    * The graphs are present along with the output. For Example in cell 4 after each run of the algorithm , the graphs are present so scrolling to the end of the output shows the graph
    
    * The code is commented to show the various steps used
    * Random seeds are used to reproduce the results
    * Due to space constraints only the 2 best graphs are shown in the report. All the rest of the graphs are visible from the output

In [ ]:
import random

# Extracting the data from the file
def extract_data_from_file():
    file = open('output.txt','r')
    content = file.readlines()
    weights = []
    values = []
    max_cost = 0
    # Extracting the first line to get the max weight constraint
    first_line = content[0].split(' ')
    for k in first_line:
        x=k.strip()
        if(x.isdigit()==True):
            max_cost = x
    for i in range(1,len(content)):
        data = content[i].split(' ')
        # Extracting the digit with the weight contents
        if('weight:' in data):
            for j in data:
                res = False
                value = j.strip()
                try:
                    float(value)
                    res = True
                except:
                    res = False
                if(res==True):
                    weights.append(value)
         # Extracting the digit with the value contents
        if('value:' in data):
            for j in data:
                value = j.strip()
                if(value.isdigit()==True):
                    values.append(value)
    weights = [float(i) for i in weights]
    values = [int(i) for i in values]
    max_cost = float(max_cost)
    return weights,values,max_cost


import matplotlib.pyplot as plt
import numpy as np
def knapsack(max_cost,value_list,cost_list,population_size,mutation_rate,tournament_size,Population):
    # History will contain best solutions with best fitness in every generation
    # avg History will contain average fitness in every generation
    History = []
    avg_History = []
    Flag = False
    return_value = 0
    population = []
    for i in Population:
        population.append(i)
    count = 1 
    # repeating for 10000 generations
    while(count<=10000):
        # tournament_size is the k value
        parents_selected = selection_from_population(population,value_list,population_size,tournament_size)
        offspring = crossover_parents(parents_selected,value_list)
        corrected_mutated_offspring =[]
        # Handling of edge case in case mutation is not required
        if(mutation_rate == 0):
            offspring_mutated = offspring
        else:
            offspring_mutated = mutate(offspring,mutation_rate)
        for i in offspring_mutated:
            # Indirect encoding correction after mutation
            corrected_mutated_offspring.append(indirect_encoding_correction(i,max_cost,cost_list,value_list))
        fitnesses_of_population = calculate_Fitness(population,value_list)
        fitnesses_of_children = calculate_Fitness(corrected_mutated_offspring,value_list)
        for i in range(0,len(fitnesses_of_children)):
            if(fitnesses_of_children[i]>min(calculate_Fitness(population,value_list))):
                # replacing the weakest offspring
                index = fitnesses_of_population.index(min(fitnesses_of_population))
                population.pop(index)
                population.append(corrected_mutated_offspring[i])     
        next_gen = population
        History.append(max(calculate_Fitness(population,value_list)))
        avg_History.append(sum(calculate_Fitness(population,value_list))/100)
        confirm_value = calculate_Fitness(population,value_list)
        print('Best Fitness in Generation {0} is {1} and weight is {2}'.format(count,max(calculate_Fitness(population,value_list)),findmax_cost(population[confirm_value.index(max(confirm_value))],max_cost,cost_list)))
        if(count==9999):
            for i in range(0,100):
                if(check_validity(population[i],max_cost,cost_list)):
                    Flag = True
                else:
                    Flag = False
                    break
            if(Flag == False):
                print('Invalid Solns Present')
            else:
                print('All OK')
            return_value = max(calculate_Fitness(population,value_list))
        count = count + 1
    plt.figure()
    print(len(History))
    plt.subplot(1,3,1);plt.scatter(list(range(1,10001)),History);plt.xlabel('Generations');plt.ylabel('Best Solution in Each Generation')
    plt.subplot(1,3,3);plt.scatter(list(range(1,10001)),avg_History);plt.xlabel('Generations');plt.ylabel('Average Solution in Each Generation')
    plt.show()
    return return_value
    
        

def create_valid_randomised_population(max_cost,cost_list,value_list,population_size):
    count = 1
    population = []
    while(count <= population_size):
        bits =''
        for i in range(0,100):
            # Random selection of bits
            bits = bits + str(np.random.randint(0,2))
        if(check_validity(bits,max_cost,cost_list) == True):
            count = count + 1
            population.append(bits)
        else:
            weight = findmax_cost(bits,max_cost,cost_list)
            # Indirect encoding mechanism. If weight > max cost then remove bags with minimum value (1 ---> 0 for bags with 
            # minimum value) to prevent value loss. Continue till weight>max weight
            while(weight>max_cost):
                index = []
                for j in range(0,len(bits)):
                    if(bits[j]=='1'):
                        index.append(j)
                values = []
                for j in index:
                    values.append(value_list[j])
                min_value = values.index(min(values))
                min_value_index = index[min_value]

                bits = bits[0:min_value_index]+'0'+bits[min_value_index+1:]
                weight = findmax_cost(bits,max_cost,cost_list)
            count = count + 1
            population.append(bits)
    return population


def check_validity(bits,max_cost,cost_list):
    flag = False
    sums = 0
    for i in range(0,len(bits)):
        value = int(bits[i])
        sums = sums + cost_list[i]*value
    if(sums <= max_cost):
        return True
    else:
        return False
        
def findmax_cost(bits,max_cost,cost_list):
    flag = False
    sums = 0
    for i in range(0,len(bits)):
        value = int(bits[i])
        sums = sums + cost_list[i]*value
        sums = round(sums,3)
    return sums

def selection_from_population(population,value_list,population_size,tournament_size):
    final_parents = []
    # first loop to be executed twice for 2 parents
    for i in range(1,3):
        parents = []
        # choosing k member indices of the tournament
        indexes = list(np.random.randint(0,100,tournament_size))
        # choosing the members and competing
        for j in range(0,tournament_size):
            parents.append(population[int(indexes[j])])
        fitness = calculate_Fitness(parents,value_list)
        index = fitness.index(max(fitness))
        final_parents.append(parents[index])
    return final_parents

def calculate_Fitness(values,value_list):
    fitness = []
    for i in values:
        sums = 0
        for j in range(0,len(i)):
            value_index = int(i[j])
            sums = sums + value_list[j]*value_index
        fitness.append(sums)
    return fitness

def crossover_parents(parents_selected,value_list):
    offspring = []
    parent_1 = parents_selected[0]
    parent_2 = parents_selected[1]
    # crossover point is random
    cross_over_point = np.random.randint(0,100)
    child_1 = parent_1[0:cross_over_point] + parent_2[cross_over_point:]
    child_2 = parent_2[0:cross_over_point] + parent_1[cross_over_point:]
    offspring.append(child_1)
    offspring.append(child_2)
    return offspring
        

def mutate(offspring,number):
    offsprings=[]
    for j in offspring:
        indexes = []
        for i in range(0,number):
            # mutation point is random
            indexes.append(np.random.randint(0,100))
        for i in indexes:
            if(j[i] == '1'):
                j = j[0:i] + '0' + j[i+1:]
            else:
                j = j[0:i] + '1' + j[i+1:]
        offsprings.append(j)
    return offsprings

# indirect encoding in a separate function . Only the correction part is written again to correct solutions
def indirect_encoding_correction(bits,max_cost,cost_list,value_list):
    while(findmax_cost(bits,max_cost,cost_list)>max_cost):
        index = []
        for j in range(0,len(bits)):
            if(bits[j]=='1'):
                index.append(j)
        values = []
        for j in index:
            values.append(value_list[j])
        min_value = values.index(min(values))
        min_value_index = index[min_value]
        if(bits[min_value_index]=='1'):
            bits = bits[0:min_value_index]+'0'+bits[min_value_index+1:]
    return bits

def begin_knapsack_problem(population_size,mutation_rate,tournament_size,population):
    return knapsack(max_cost,values,weights,population_size,mutation_rate,tournament_size,population)

In [ ]:
import random
import numpy as np

#### Creating a valid Random Population of Size 100 and using a Random seed to Reproduce result

In [ ]:
np.random.seed(9)
weights,values,max_cost = extract_data_from_file()
Population = create_valid_randomised_population(max_cost,weights,values,population_size=100)

In [ ]:
np.random.seed(999)
weights,values,max_cost = extract_data_from_file()
Population_2 = create_valid_randomised_population(max_cost,weights,values,population_size=200)

#### Iteration 1 : Population size = 100 , Mutation Rate = 1 , Tournament size = 2 , crossover = included

In [ ]:
np.random.seed(50)
data = []
for i in range(3):
    data.append(begin_knapsack_problem(100,1,2,Population))
print(sum(data)/len(data))

#### Iteration 2 : Population size = 100 , Mutation Rate = 2 , Tournament size = 2 , crossover = included

In [ ]:
np.random.seed(89)
data_2 = []
for i in range(3):
    data_2.append(begin_knapsack_problem(100,2,2,Population))
print(sum(data_2)/len(data_2))

#### Iteration 3 : Population size = 100 , Mutation Rate = 3 , Tournament size = 2 , crossover = included

In [ ]:
np.random.seed(91)
data_3 = []
for i in range(3):
    data_3.append(begin_knapsack_problem(100,3,2,Population))
print(print(sum(data_3)/len(data_3)))

#### Iteration 4 : Population size = 100 , Mutation Rate = 1 , Tournament size = 20 , crossover = included

In [ ]:
np.random.seed(53)
data_4 = []
for i in range(3):
    data_4.append(begin_knapsack_problem(100,1,20,Population))
print(sum(data_4)/len(data_4))

#### Iteration 5 : Population size = 100 , Mutation Rate = 2 , Tournament size = 20 , crossover = included

In [ ]:
np.random.seed(54)
data_5=[]
for i in range(3):
    data_5.append(begin_knapsack_problem(100,2,20,Population))
print(sum(data_5)/len(data_5))

#### Iteration 6 : Population size = 100 , Mutation Rate = 3 , Tournament size = 20 , crossover = included

In [ ]:
np.random.seed(92)
data_6=[]
for i in range(3):
    data_6.append(begin_knapsack_problem(100,3,20,Population))
print(sum(data_6)/len(data_6))

#### Iteration 7 : Population size = 100 , Mutation Rate = 3 , Tournament size = 40 , crossover = included

In [ ]:
np.random.seed(56)
data_7 = []
for i in range(3):
    data_7.append(begin_knapsack_problem(100,3,40,Population))
print(sum(data_7)/len(data_7))

#### Iteration 8 : Population size = 100 , Mutation Rate = 2 , Tournament size = 40 , crossover = included

In [ ]:
np.random.seed(57)
data_8 = []
for i in range(3):
    data_8.append(begin_knapsack_problem(100,2,40,Population))
print(sum(data_8)/len(data_8))

#### Iteration 9 : Population size = 100 , Mutation Rate = 1 , Tournament size = 40 , crossover = included

In [ ]:
np.random.seed(58)
data_9 = []
for i in range(3):
    data_9.append(begin_knapsack_problem(100,1,40,Population))
print(sum(data_9)/len(data_9))

#### Iteration 10 : Population size = 100 , Mutation Rate = 1 , Tournament size = 4 , crossover = included

In [ ]:
np.random.seed(96)
data_10 = []
for i in range(3):
    data_10.append(begin_knapsack_problem(100,1,4,Population))
print(sum(data_10)/len(data_10))

#### Iteration 11 : Population size = 100 , Mutation Rate = 2 , Tournament size = 4 , crossover = included

In [ ]:
np.random.seed(60)
data_11 = []
for i in range(3):
    data_11.append(begin_knapsack_problem(100,2,4,Population))
print(sum(data_11)/len(data_11))

#### Iteration 12 : Population size = 100 , Mutation Rate = 3 , Tournament size = 4 , crossover = included

In [ ]:
np.random.seed(61)
data_12 = []
for i in range(3):
    data_12.append(begin_knapsack_problem(100,3,4,Population))
print(sum(data_12)/len(data_12))

#### Iteration 13 : Population size = 100 , Mutation Rate = 1 , Tournament size = 3 , crossover = included

In [ ]:
np.random.seed(65)
data_13 = []
for i in range(3):
    data_13.append(begin_knapsack_problem(100,1,3,Population))
print(sum(data_13)/len(data_13))

#### Iteration 14 : Population size = 100 , Mutation Rate = 2 , Tournament size = 3 , crossover = included

In [ ]:
np.random.seed(86)
data_14 = []
for i in range(3):
    data_14.append(begin_knapsack_problem(100,2,3,Population))
print(sum(data_14)/len(data_14))

#### Iteration 15 : Population size = 100 , Mutation Rate = 3 , Tournament size = 3 , crossover = included

In [ ]:
np.random.seed(87)
data_15 = []
for i in range(3):
    data_15.append(begin_knapsack_problem(100,3,3,Population))
print(sum(data_15)/len(data_15))

#### Iteration 16 : population size 200 , mutation rate 1 , tournament selection 2

In [ ]:
np.random.seed(1000)
data_200 = []
for i in range(3):
    data_200.append(begin_knapsack_problem(200,1,2,Population_2))
print(sum(data_200)/len(data_200))

#### Iteration 18 : population size 200 , Tournament size 3, mutation rate = 2

In [ ]:
np.random.seed(1001)
data_201 = []
for i in range(3):
    data_201.append(begin_knapsack_problem(200,2,2,Population_2))
print(sum(data_201)/len(data_201))

#### Iteration 19 : population size 200 , Tournament size 2, mutation rate = 3

In [ ]:
np.random.seed(1002)
data_202 = []
for i in range(3):
    data_202.append(begin_knapsack_problem(200,3,2,Population_2))
print(sum(data_202)/len(data_202))

#### Iteration 20 : Population = 200, Mutation rate = 1, k = 3

In [ ]:
np.random.seed(1003)
data_203 = []
for i in range(3):
    data_203.append(begin_knapsack_problem(200,1,3,Population_2))
print(sum(data_203)/len(data_203))

#### Iteration 21 : Population = 200, Mutation rate = 2, k = 3

In [ ]:
np.random.seed(1004)
data_204 = []
for i in range(3):
    data_204.append(begin_knapsack_problem(200,2,3,Population_2))
print(sum(data_204)/len(data_204))

#### Iteration 22 : Population = 200, Mutation rate = 3, k = 3

In [ ]:
np.random.seed(1005)
data_205 = []
for i in range(3):
    data_205.append(begin_knapsack_problem(200,3,3,Population_2))
print(sum(data_205)/len(data_205))

## Results Obtained

In [ ]:
print('Combination 1 (Tournament size = {0} , Mutation Rate = {1}) : Best Fitness = {2} and Average Fitness ={3}'.format(2,1,max(data),sum(data)/len(data)))
print('Combination 2 (Tournament size = {0} , Mutation Rate = {1}) : Best Fitness = {2} and Average Fitness ={3}'.format(2,2,max(data_2),sum(data_2)/len(data_2)))
print('Combination 3 (Tournament size = {0} , Mutation Rate = {1}) : Best Fitness = {2} and Average Fitness ={3}'.format(2,3,max(data_3),sum(data_3)/len(data_3)))
print('Combination 4 (Tournament size = {0} , Mutation Rate = {1}) : Best Fitness = {2} and Average Fitness ={3}'.format(20,1,max(data_4),sum(data_4)/len(data_4)))
print('Combination 5 (Tournament size = {0} , Mutation Rate = {1}) : Best Fitness = {2} and Average Fitness ={3}'.format(20,2,max(data_5),sum(data_5)/len(data_5)))
print('Combination 6 (Tournament size = {0} , Mutation Rate = {1}) : Best Fitness = {2} and Average Fitness ={3}'.format(20,3,max(data_6),sum(data_6)/len(data_6)))
print('Combination 7 (Tournament size = {0} , Mutation Rate = {1}) : Best Fitness = {2} and Average Fitness ={3}'.format(40,1,max(data_9),sum(data_9)/len(data_9)))
print('Combination 8 (Tournament size = {0} , Mutation Rate = {1}) : Best Fitness = {2} and Average Fitness ={3}'.format(40,2,max(data_8),sum(data_8)/len(data_8)))
print('Combination 9 (Tournament size = {0} , Mutation Rate = {1}) : Best Fitness = {2} and Average Fitness ={3}'.format(40,3,max(data_7),sum(data_7)/len(data_7)))


In [ ]:
print('Combination 10 (Tournament size = {0} , Mutation Rate = {1}) : Best Fitness = {2} and Average Fitness ={3}'.format(4,1,max(data_10),sum(data_10)/len(data_10)))
print('Combination 11 (Tournament size = {0} , Mutation Rate = {1}) : Best Fitness = {2} and Average Fitness ={3}'.format(4,2,max(data_11),sum(data_11)/len(data_11)))
print('Combination 12 (Tournament size = {0} , Mutation Rate = {1}) : Best Fitness = {2} and Average Fitness ={3}'.format(4,3,max(data_12),sum(data_12)/len(data_12)))
print('Combination 13 (Tournament size = {0} , Mutation Rate = {1}) : Best Fitness = {2} and Average Fitness ={3}'.format(3,1,max(data_13),sum(data_13)/len(data_13)))
print('Combination 14 (Tournament size = {0} , Mutation Rate = {1}) : Best Fitness = {2} and Average Fitness ={3}'.format(3,2,max(data_14),sum(data_14)/len(data_14)))
print('Combination 15 (Tournament size = {0} , Mutation Rate = {1}) : Best Fitness = {2} and Average Fitness ={3}'.format(3,3,max(data_15),sum(data_15)/len(data_15)))


#### Supplimentary Iteration 1 : Removal of Mutation from Iteration 2

In [ ]:
np.random.seed(71)
data_2 = []
for i in range(3):
    data_2.append(begin_knapsack_problem(100,0,2,Population))
print(sum(data_2)/len(data_2))

#### Supplimentary Iteration 2 : Removal of Crossover from the code and application on Iteration 2

In [ ]:
import random
def extract_data_from_file():
    file = open('output.txt','r')
    content = file.readlines()
    weights = []
    values = []
    max_cost = 0
    first_line = content[0].split(' ')
    for k in first_line:
        x=k.strip()
        if(x.isdigit()==True):
            max_cost = x
    for i in range(1,len(content)):
        data = content[i].split(' ')
        if('weight:' in data):
            for j in data:
                res = False
                value = j.strip()
                try:
                    float(value)
                    res = True
                except:
                    res = False
                if(res==True):
                    weights.append(value)
        if('value:' in data):
            for j in data:
                value = j.strip()
                if(value.isdigit()==True):
                    values.append(value)
    weights = [float(i) for i in weights]
    values = [int(i) for i in values]
    max_cost = float(max_cost)
    return weights,values,max_cost


import matplotlib.pyplot as plt
import numpy as np
def knapsack_remove_crossover(max_cost,value_list,cost_list,population_size,mutation_rate,tournament_size,Population):
    History = []
    avg_History = []
    Flag = False
    return_value = 0
    #print('Stuck 0')
    #population = create_valid_randomised_population(max_cost,cost_list,value_list,population_size)
    population = []
    for i in Population:
        population.append(i)
    count = 1 
    
    while(count<=10000):
        #print('Stuck 1')
        parents_selected = selection_from_population(population,value_list,population_size,tournament_size)
        #print('Stuck 2')
        offspring = parents_selected#crossover_parents(parents_selected,value_list)
        #print('length of cross over is {0}'.format(len(offspring)))
        corrected_mutated_offspring =[]
        #print('Stuck 3') 
        #print('Stuck 4')
        if(mutation_rate == 0):
            offspring_mutated = offspring
        else:
            offspring_mutated = mutate(offspring,mutation_rate)
        #print('Stuck 5')
        for i in offspring_mutated:
            corrected_mutated_offspring.append(indirect_encoding_correction(i,max_cost,cost_list,value_list))
        
        #population = offspring
        #print('length of mutated is {0}'.format(len(corrected_mutated_offspring)))
        fitnesses_of_population = calculate_Fitness(population,value_list)
        #print('Stuck 7')
        fitnesses_of_children = calculate_Fitness(corrected_mutated_offspring,value_list)
        for i in range(0,len(fitnesses_of_children)):
            if(fitnesses_of_children[i]>min(calculate_Fitness(population,value_list))):
                #print('Generation : {0} Child Fitness : {1} replacing parent Fitness {1} '.format(count,fitnesses_of_children[i],min(fitnesses_of_population)))
                index = fitnesses_of_population.index(min(fitnesses_of_population))
                population.pop(index)
                population.append(corrected_mutated_offspring[i])     
        next_gen = population
        History.append(max(calculate_Fitness(population,value_list)))
        avg_History.append(sum(calculate_Fitness(population,value_list))/100)
        print('Best Fitness in Generation {0} is {1}'.format(count,max(calculate_Fitness(population,value_list))))
        if(count==9999):
            for i in range(0,100):
                if(check_validity(population[i],max_cost,cost_list)):
                    Flag = True
                else:
                    Flag = False
                    break
            if(Flag == False):
                print('Invalid Solns Present')
            else:
                print('All OK')
            return_value = max(calculate_Fitness(population,value_list))
            #print('Best Fitness in Generation {0} is {1}'.format(count,return_value))
        count = count + 1
    plt.figure()
    print(len(History))
    plt.subplot(1,3,1);plt.scatter(list(range(1,10001)),History);plt.xlabel('Generations');plt.ylabel('Best Solution in Each Generation')
    plt.subplot(1,3,3);plt.scatter(list(range(1,10001)),avg_History);plt.xlabel('Generations');plt.ylabel('Average Solution in Each Generation')
    plt.show()
    return return_value

def begin_knapsack_problem_without_crossover(population_size,mutation_rate,tournament_size,population):
    return knapsack_remove_crossover(max_cost,values,weights,population_size,mutation_rate,tournament_size,population)

In [ ]:
np.random.seed(82)
data_20 = []
for i in range(3):
    data_20.append(begin_knapsack_problem_without_crossover(100,2,2,Population))
print(sum(data_20)/len(data_20))

In [ ]:
# Further experiment on duplicates in higher K values of tournament selection
# here for values of k = 3 and k=20 the indexes of parents (0-100) are chosen.
# We are checking duplicates to determine whether a very high selection pressure due to large value of k 
# leads to choosing the same indivisuals in a population

for i in range(20):
    x=[]
    y=[]
    for i in range(3):
        x.append(np.random.randint(0,100))
    for i in range(20):
        y.append(np.random.randint(0,100))
    print('number of duplicates with k = 3 is {0}'.format((len(x)-len(set(x)))))
    print('number of duplicates with k = 20 is {0}'.format((len(y)-len(set(y)))))